In [1]:
import pandas as pd

In [12]:
df = pd.read_csv('2019_Movies.csv')
df.head(10)

,Movie_name,Actors,Crtitic_ratings,Release date & duration,User_ratings,Language,Movie_type
0,Good Newwz,"Akshay Kumar, Kareena Kapoor, Diljit Dosanjh, ...",3.5,27 Dec 2019 | 2 hrs 14 mins,3.4,Hindi,"Comedy, Drama | UA"
1,Dabangg 3,"Salman Khan, Sonakshi Sinha, Arbaaz Khan, Kich...",3.0,20 Dec 2019 | 2 hrs 23 mins,3.2,"Hindi, Telugu, Kannada, Tamil","Action, Romance, Thriller | UA"
2,Sound Of Water,"Ramesh Goyal, Chetan Sharma, Abraam Pandey, Ja...",3.4,13 Dec 2019 | 2 hrs 25 mins,NaN,Hindi,Drama
3,The Body,"Emraan Hashmi, Rishi Kapoor, Vedhika, Sobhita ...",2.5,13 Dec 2019 | 2 hrs 15 mins,2.8,Hindi,Drama | UA
4,Mudda 370 J&K,"Hiten Tejwani, Manoj Joshi, Raj Zutshi, Zarina...",1.0,13 Dec 2019 | 2 hrs 12 mins,1.7,Hindi,Drama | A
5,Kanoon Ke Khiladi,"Hemant Birje, Gulshan Pandey, Anil Yadav",2.8,13 Dec 2019 | 2 hrs 6 mins,NaN,Hindi,Drama | UA
6,Unfriends,"Rajat Rawail, Himayat Ali, Mousumi Biswas, Naf...",1.0,13 Dec 2019 | 2 hrs 10 mins,1.7,Hindi,Drama | UA
7,Mardaani 2,"Rani Mukerji, Shruti Bapna, Rajesh Sharma, Jis...",3.5,13 Dec 2019 | 2 hrs 2 mins,3.4,Hindi,"Action, Crime, Thriller | UA"
8,Panipat,"Sanjay Dutt, Kriti Sanon, Arjun Kapoor, Padmin...",3.5,06 Dec 2019 | 2 hrs 53 mins,3.3,Hindi,"Drama, Action, History | U"
9,A Gift Of Love: Sifar,"Sudha Chandran, Kanikka Kapur, Mushtaq Khan, A...",1.5,06 Dec 2019 | 1 hr 39 mins,NaN,Hindi,"Drama,Romantic | UA"


In [13]:
df.describe

<bound method NDFrame.describe of                      Movie_name  \
0                    Good Newwz   
1                     Dabangg 3   
2                Sound Of Water   
3                      The Body   
4                 Mudda 370 J&K   
..                          ...   
219    URI: The Surgical Strike   
220                 Salt Bridge   
221                 Dharma Bhai   
222            Kanpur Ke Sholay   
223  English Ki Taay Taay Fisss   

                                                Actors  Crtitic_ratings  \
0    Akshay Kumar, Kareena Kapoor, Diljit Dosanjh, ...              3.5   
1    Salman Khan, Sonakshi Sinha, Arbaaz Khan, Kich...              3.0   
2    Ramesh Goyal, Chetan Sharma, Abraam Pandey, Ja...              3.4   
3    Emraan Hashmi, Rishi Kapoor, Vedhika, Sobhita ...              2.5   
4    Hiten Tejwani, Manoj Joshi, Raj Zutshi, Zarina...              1.0   
..                                                 ...              ...   
219  Vicky Kaushal, 

In [14]:
df.columns

Index(['Movie_name', 'Actors', 'Crtitic_ratings', 'Release date & duration',
       'User_ratings', 'Language', 'Movie_type'],
      dtype='object')

In [18]:
C = df['Crtitic_ratings'].mean()
print(C)

2.793273542600897


In [19]:
m =df['Crtitic_ratings'].quantile(0.90)
print(m)

3.7


In [20]:
q_movies = df.copy().loc[df['Crtitic_ratings'] >= m]
q_movies.shape

(24, 7)

In [27]:
def weighted_rating(x, m=m, C=C):
    v = x['User_ratings']
    R = x['Crtitic_ratings']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [22]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [23]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [28]:
q_movies[['Movie_name', 'User_ratings', 'Crtitic_ratings', 'score']].head(15)

,Movie_name,User_ratings,Crtitic_ratings,score
29,Bala,4.2,4.0,3.434824
32,Ekta,NaN,4.2,NaN
62,Section 375,4.0,4.0,3.420144
86,Meri Bhagya Laxmi,NaN,3.7,NaN
100,Article 15,4.3,4.0,3.441889
158,Kesari,4.5,4.0,3.455501
160,The Pushkar Lodge,NaN,3.8,NaN
161,Bindi,NaN,3.7,NaN
167,Hamid,3.7,4.0,3.396637
173,Suspense,NaN,3.7,NaN


In [37]:
df['Movie_name'].head()

0        Good Newwz
1         Dabangg 3
2    Sound Of Water
3          The Body
4     Mudda 370 J&K
Name: Movie_name, dtype: object

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
df['Movie_namee'] = df['Movie_name'].fillna('')

In [33]:
tfidf_matrix = tfidf.fit_transform(df['Movie_name'])

In [35]:
tfidf_matrix.shape

(14332, 13602)

In [34]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [29]:
indices = pd.Series(df.index, index=df['Movie_name']).drop_duplicates()

In [36]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    movie_indices = [i[0] for i in sim_scores]

    return df['Movie_name'].iloc[movie_indices]

In [38]:
get_recommendations('Good Newwz')

1                 Dabangg 3
2            Sound Of Water
3                  The Body
4             Mudda 370 J&K
5         Kanoon Ke Khiladi
6                 Unfriends
7                Mardaani 2
8                   Panipat
9     A Gift Of Love: Sifar
10       Pati Patni Aur Woh
Name: Movie_name, dtype: object